# YT Comments analysis

In [10]:
from googleapiclient.discovery import build
import pandas as pd
from api import API_KEY

channel_id = "UCWeg2Pkate69NFdBeuRFTAw" #Squeezie channel

youtube = build('youtube', 'v3', developerKey=API_KEY)

exemple_video = "qCKyRhkhqoQ"

In [6]:
def get_channel_stats(youtube, channel_id):
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id
    )
    response = request.execute()
    data = {
        "channel_name": response['items'][0]['snippet']['title'],
        "channel_id": response['items'][0]['id'],
        "stats": response['items'][0]['statistics']
    }
    return data

In [8]:
def get_video_comments(youtube,video_Id):
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_Id,
        maxResults=1000,
    )
    response = request.execute()
    return response

In [14]:
def get_video_comments_texts(youtube, video_Id):
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_Id,
        maxResults=1000,
    )
    response = request.execute()
    
    # Extract textOriginal from each item
    comments_texts = []
    for item in response.get("items", []):
        if "snippet" in item and "topLevelComment" in item["snippet"]:
            comment_snippet = item["snippet"]["topLevelComment"]["snippet"]
            if "textOriginal" in comment_snippet:
                comments_texts.append(comment_snippet["textOriginal"])
    
    return comments_texts

In [22]:
get_channel_stats(youtube, channel_id)
video = get_video_comments_texts(youtube, exemple_video)

In [72]:
import pandas as pd
import re
from cleantext import clean

pd.set_option('display.max_rows', False)

df = pd.DataFrame({"Com":video})

df

,Com
0,Je suis en fou rire sur le calendrier
1,Zafeel en octobre c'est quelque chose 😭😭
2,On peut encore t’envoyer des colis ?
3,Hello
4,Hello
5,La masterclass des gage c'est trop 😂😂😂
6,Mastu qui est tendu toute les 10sec 😅
7,le pauvre Harry
8,*oééééééé super!!!une video de toi et les poto...
...,...


In [73]:
from langdetect import detect, LangDetectException

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

df["Language"] = df["Com"].apply(detect_language)
df

,Com,Language
0,Je suis en fou rire sur le calendrier,fr
1,Zafeel en octobre c'est quelque chose 😭😭,fr
2,On peut encore t’envoyer des colis ?,fr
3,Hello,fi
4,Hello,fi
5,La masterclass des gage c'est trop 😂😂😂,fr
6,Mastu qui est tendu toute les 10sec 😅,fr
7,le pauvre Harry,fr
8,*oééééééé super!!!une video de toi et les poto...,fr
...,...,...
